## Inspect Error Tag Distributions of GEC Datasets

In [1]:
import os.path

### BEA19

In [2]:
DATA_DIR = "datasets_train/wi+locness/m2/"
tag_list = ["ADJ","ADJ:FORM","ADV","CONJ","CONTR","DET","K","MORPH","NOUN","NOUN:INFL","NOUN:NUM","NOUN:POSS","ORTH","OTHER","PART","PREP","PRON","PUNCT","SPELL","VERB","VERB:FORM","VERB:INFL","VERB:SVA","VERB:TENSE","WO"]
tag_count_dict = dict.fromkeys(tag_list, 0)
tag_count_dict["noop"] = 0
print(tag_count_dict)

{'ADJ': 0, 'ADJ:FORM': 0, 'ADV': 0, 'CONJ': 0, 'CONTR': 0, 'DET': 0, 'K': 0, 'MORPH': 0, 'NOUN': 0, 'NOUN:INFL': 0, 'NOUN:NUM': 0, 'NOUN:POSS': 0, 'ORTH': 0, 'OTHER': 0, 'PART': 0, 'PREP': 0, 'PRON': 0, 'PUNCT': 0, 'SPELL': 0, 'VERB': 0, 'VERB:FORM': 0, 'VERB:INFL': 0, 'VERB:SVA': 0, 'VERB:TENSE': 0, 'WO': 0, 'noop': 0}


In [3]:
def parse_error_tags(filepath):
    with open(filepath, "r") as f:
        for line in f:
            if line.startswith('A'):
                error_tag = line.split('|||')[1]
                if error_tag != "noop":
                    tag_count_dict[error_tag[2:]] += 1
                else:
                    tag_count_dict["noop"] += 1

In [4]:
for file in os.listdir(DATA_DIR):
    filepath = os.path.join(DATA_DIR, file)
    parse_error_tags(filepath)
print(tag_count_dict)

{'ADJ': 2164, 'ADJ:FORM': 338, 'ADV': 2152, 'CONJ': 736, 'CONTR': 440, 'DET': 15922, 'K': 3640, 'MORPH': 2674, 'NOUN': 6208, 'NOUN:INFL': 176, 'NOUN:NUM': 5662, 'NOUN:POSS': 892, 'ORTH': 6782, 'OTHER': 18214, 'PART': 1194, 'PREP': 13946, 'PRON': 3712, 'PUNCT': 24814, 'SPELL': 5542, 'VERB': 8270, 'VERB:FORM': 5008, 'VERB:INFL': 64, 'VERB:SVA': 3136, 'VERB:TENSE': 8672, 'WO': 2272, 'noop': 25582}


In [5]:
# calculate probability of each tag
tag_prob_dict = dict.fromkeys(tag_list, 0.0)
tag_prob_dict["noop"] = 0.0
tag_count_sum = sum(tag_count_dict.values())
for tag in tag_count_dict.keys():
    tag_prob_dict[tag] = tag_count_dict[tag] / tag_count_sum
print(tag_prob_dict)


{'ADJ': 0.012864718331629135, 'ADJ:FORM': 0.0020093691294319075, 'ADV': 0.01279337978265522, 'CONJ': 0.004375431003733384, 'CONTR': 0.0026157467957101752, 'DET': 0.09465436473022139, 'K': 0.02163935985542054, 'MORPH': 0.015896606663020472, 'NOUN': 0.03690580933583811, 'NOUN:INFL': 0.00104629871828407, 'NOUN:NUM': 0.03365990535752503, 'NOUN:POSS': 0.005302832140394264, 'ORTH': 0.04031816992842366, 'OTHER': 0.10828002758423894, 'PART': 0.00709818562290443, 'PREP': 0.08290728366585023, 'PRON': 0.022067391149264023, 'PUNCT': 0.14751622951989157, 'SPELL': 0.03294651986778589, 'VERB': 0.049164150001188976, 'VERB:FORM': 0.029771954438446723, 'VERB:INFL': 0.0003804722611942073, 'VERB:SVA': 0.01864314079851616, 'VERB:TENSE': 0.05155399139181509, 'WO': 0.01350676527239436, 'noop': 0.15208189665422206}
